[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/falcon/falcon-40b-chatbot.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/falcon/falcon-40b-chatbot.ipynb)

In [1]:
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00


## Initializing the Hugging Face Pipeline

The first thing we need to do is initialize a `text-generation` pipeline with Hugging Face transformers. The Pipeline requires three things that we must initialize first, those are:

* A LLM, in this case it will be `tiiuae/falcon-40b-instruct`.

* The respective tokenizer for the model.

* A stopping criteria object.

We'll explain these as we get to them, let's begin with our model.

We initialize the model and move it to our CUDA-enabled GPU. Using Colab this can take 5-10 minutes to download and initialize the model.

In [2]:
from torch import cuda, bfloat16
import transformers

model_name = 'tiiuae/falcon-7b-instruct'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto'
)
model.eval()
print(f"Model loaded on {device}")

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


The pipeline requires a tokenizer which handles the translation of human readable plaintext to LLM readable token IDs. The Falcon-40B model was trained using the `falcon-40b` tokenizer, which we initialize like so:

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

Finally we need to define the _stopping criteria_ of the model. The stopping criteria allows us to specify *when* the model should stop generating text. If we don't provide a stopping criteria the model just goes on a bit of a tangent after answering the initial question.

In [4]:
from transformers import StoppingCriteria, StoppingCriteriaList

# we create a list of stopping criteria
stop_token_ids = [
    tokenizer.convert_tokens_to_ids(x) for x in [
        ['Human', ':'], ['AI', ':']
    ]
]

stop_token_ids

[[23431, 37], [17362, 37]]

We need to convert these into `LongTensor` objects:

In [5]:
import torch

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

[tensor([23431,    37], device='cuda:0'),
 tensor([17362,    37], device='cuda:0')]

We can do a quick spot check that no `<unk>` token IDs (`0`) appear in the `stop_token_ids` — there are none so we can move on to building the stopping criteria object that will check whether the stopping criteria has been satisfied — meaning whether any of these token ID combinations have been generated.

In [6]:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

Now we're ready to initialize the HF pipeline. There are a few additional parameters that we must define here. Comments explaining these have been included in the code.

In [7]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

Confirm this is working:

In [9]:
res = generate_text("Generate for me a cover letter for my data science job.")
print(res[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Generate for me a cover letter for my data science job.
Dear [Hiring Manager],

I am writing to apply for the position of Data Scientist at [Company Name]. With a degree in Mathematics and a specialization in Data Science, I have the technical skills and analytical abilities necessary to excel in this role.

Throughout my academic and professional career, I have been exposed to a variety of data-driven problem-solving techniques, including machine learning, data mining, and statistical analysis. I have also gained experience with a range of programming languages, such as Python, R, and SQL.

In my current role as a Data Analyst, I have been responsible for developing data-driven solutions to complex business problems. I have also been involved in creating data-driven dashboards and reports to help our clients make informed decisions.

I am confident that my skills and experience make me an ideal candidate for this position. I look forward to discussing the opportunity further during an

Now to implement this in LangChain

In [10]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

# template for an instruction with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}"
)

llm = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [11]:
print(llm_chain.predict(
    instruction="Generate for me a cover letter for my data science job."
).lstrip())

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Dear [Hiring Manager],

I am writing to apply for the position of Data Scientist at [Company Name]. With a degree in Mathematics and a specialization in Data Science, I have the technical skills and analytical abilities necessary to excel in this role.

Throughout my academic and professional career, I have been exposed to a variety of data-driven problem-solving techniques, including machine learning, data mining, and statistical analysis. I have also gained experience with a range of programming languages, such as Python, R, and SQL.

In my current role as a Data Analyst, I have been responsible for developing data-driven solutions to complex business problems. I have also been involved in creating data-driven dashboards and reports to help our clients make informed decisions.

I am confident that my skills and experience make me an ideal candidate for this position. I look forward to discussing the opportunity further during an interview.

Thank you for your consideration.

Sincerel

We still get the same output as we're not really doing anything differently here, but we have now added MTP-30B-chat to the LangChain library. Using this we can now begin using LangChain's advanced agent tooling, chains, etc, with MTP-30B.

## Falcon-40B Chatbot

Using the above and LangChain we can create a conversational agent very easily. We start by initializing the conversational memory required:

In [12]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    memory_key="history",  # important to align with agent prompt (below)
    k=5,
    #return_messages=True  # for conversation agent
    return_only_outputs=True  # for conversation chain
)
#k=5 means it will keep track of the most recent 5 entries
#we may need to adjust this depending on number of questions we are planning to ask user

Now we initialize the conversational chain itself:

In [13]:
from langchain.chains import ConversationChain

chat = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [14]:
res = chat.predict(input='hi how are you?')
res

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi how are you?
AI:

> Finished chain.


" hi, I'm doing great. How about you?\n\nAI:"

Looks good so far, but there's a clear issue here, our output includes the cut off we set of `"Human:"`. Naturally we don't want to include this in the output we're returning to a user. We can parse this out manually or we can modify our prompt template to include an **output parser**.

To do this we will first need to create our output parser, which we do like so:

In [15]:
from langchain.schema import BaseOutputParser

class OutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        """Cleans output text"""
        text = text.strip()
        # remove suffix containing "Human:" or "AI:"
        stopwords = ['Human:', 'AI:']
        for word in stopwords:
            text = text.removesuffix(word)
        return text.strip()

    @property
    def _type(self) -> str:
        """Return output parser type for serialization"""
        return "output_parser"

parser = OutputParser()

Now we initialize a new prompt template, for that we need to initialize the object with a conversational prompt template, we can re-use our existing one from the conversational chain.

In [16]:
print(chat.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [17]:
prompt_template = \
"""The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:"""

prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=prompt_template,
    output_parser=parser
)

memory = ConversationBufferWindowMemory(
    memory_key="history",  # important to align with agent prompt (below)
    k=5,
    #return_messages=True  # for conversation agent
    return_only_outputs=True  # for conversation chain
)

chat = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True,
    prompt=prompt
)

With everything initialized we can try `predict_and_parse` which will predict for our model and then parse that prediction through the output parser we have defined.

In [18]:
res = chat.predict_and_parse(input='hi how are you?')
res

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi how are you?
AI:

> Finished chain.


"hi, I'm doing great. How about you?"

Now things are working and we don't have the messy `"Human:"` string left at the end of our returned output. Naturally we can add more logic as needed to the output parser.

We can continue the conversation to see how well Falcon 40B performs...

In [19]:
query = \
"""can you write me a simple cover letter that highlights my skills in python and tableau"""

res = chat.predict_and_parse(input=query)
res

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi how are you?
AI:  hi, I'm doing great. How about you?

AI:
Human: can you write me a simple cover letter that highlights my skills in python and tableau
AI:

> Finished chain.


"Sure, I can help you with that. Here's a simple cover letter that highlights your skills in Python and Tableau:\n\n\nDear [Hiring Manager],\n\nI am writing to apply for the position of [Position] at [Company Name]. With a degree in [Degree] and over [Years] of experience in data analysis, I am confident that I can contribute to your team.\n\nI have a strong background in Python programming, which I believe makes me a great candidate for this role. I have worked with Python on a variety of data analysis projects, including cleaning and transforming data, creating interactive dashboards, and developing machine learning models.\n\nIn addition to Python, I am also proficient in Tableau, a powerful visualization tool. I have used Tableau to create complex visualizations and dashboards that help businesses make data-driven decisions.\n\nI am excited to join your team and bring my skills and experience to the table. I look forward to hearing from you soon.\n\nThank you for your time and cons

In [22]:
query = \
"""i also want to add my skills in R and JMP"""

res = chat.predict_and_parse(input=query)
res

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi how are you?
AI:  hi, I'm doing great. How about you?

AI:
Human: can you write me a simple cover letter that highlights my skills in python and tableau
AI: 

Sure, I can help you with that. Here's a simple cover letter that highlights your skills in Python and Tableau:


Dear [Hiring Manager],

I am writing to apply for the position of [Position] at [Company Name]. With a degree in [Degree] and over [Years] of experience in data analysis, I am confident that I can contribute to your team.

I have a strong background in Python programming, which I believe makes me a great candidate for this role. I have worked with Python on a variety of data analysis project

"Sure, I can help you with that. Here's a simple cover letter that highlights your skills in R and JMP:\n\n\nDear [Hiring Manager],\n\nI am writing to apply for the position of [Position] at [Company Name]. With a degree in [Degree] and over [Years] of experience in data analysis, I am confident that I can contribute to your team.\n\nI have a strong background in data analysis, which I believe makes me a great candidate for this role. I have worked with R, a powerful programming language for data analysis, on a variety of data analysis projects, including cleaning and transforming data, creating interactive dashboards, and developing machine learning models.\n\nIn addition to R, I am also proficient in JMP, a powerful data visualization tool. I have used JMP to create complex visualizations and dashboards that help businesses make data-driven decisions.\n\nI am excited to join your team and bring my skills and experience to the table. I look forward to hearing from you soon.\n\nSincere

In [23]:
print(res)

Sure, I can help you with that. Here's a simple cover letter that highlights your skills in R and JMP:


Dear [Hiring Manager],

I am writing to apply for the position of [Position] at [Company Name]. With a degree in [Degree] and over [Years] of experience in data analysis, I am confident that I can contribute to your team.

I have a strong background in data analysis, which I believe makes me a great candidate for this role. I have worked with R, a powerful programming language for data analysis, on a variety of data analysis projects, including cleaning and transforming data, creating interactive dashboards, and developing machine learning models.

In addition to R, I am also proficient in JMP, a powerful data visualization tool. I have used JMP to create complex visualizations and dashboards that help businesses make data-driven decisions.

I am excited to join your team and bring my skills and experience to the table. I look forward to hearing from you soon.

Sincerely,
[Your Name

In [ ]:
# note (WQ): I DID NOT TRY RUNNING ANY OF THE CODES AFTER THIS COS IT'S NOT RELEVANT FOR US (but still left it in for future reference)

Let's try this code:

In [21]:
def circumference_of_circle(r):
    return 2*pi*r

circumference_of_circle(3)

NameError: ignored

Let's return this error back to the chatbot.

In [ ]:
query = \
"""Using this code I get the error:

`NameError: name 'pi' is not defined`

How can I fix this?"""
res = chat.predict_and_parse(input=query)
print(res)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi how are you?
AI:  I am doing well, thank you for asking! How are you?
Human:
Human: can you write me a simple Python script that calculates the circumference
of a circle given a radius `r`
AI:  Sure, here's a Python script that calculates the circumference of a circle given a radius `r`:

```python
def circumference_of_circle(r):
    return 2*pi*r
```

You can use this function by calling it with the radius as an argument, like so:

```python
circumference_of_circle(3)  # returns 18.84955...
```

Hope that helps!
Human: Using this code I get the error:

`NameError: name 'pi' is not defined`

How can I fix this?
AI:

> Finished chain.
You need to import the math module at the 

Let's try:

In [ ]:
import math

def circumference_of_circle(r):
    return 2*math.pi*r

circumference_of_circle(3)

18.84955592153876

Perfect, we got the answer — not immediately but we did get it in the end. Now let's try refactoring some code to see how the model does.

In [ ]:
def sum_numbers(n):
    total = 0
    for i in range(1, n+1):
        if i % 2 == 0:
            total += i
        else:
            total += i
    return total

# Test the function
result = sum_numbers(10)
print(result)

55


In [ ]:
def sum_numbers(n):
    total = 0
    for i in range(2, n+1, 2):
        total += i
    return total

# Test the function
result = sum_numbers(10)
print(result)


30


In [ ]:
query = \
"""Thanks that works! I have some code that I'd like to refactor, can you help?

The code is:

```python
def sum_numbers(n):
    total = 0
    for i in range(1, n+1):
        if i % 2 == 0:
            total += i
        else:
            total += i
    return total

# Test the function
result = sum_numbers(10)
print(result)
```
"""

res = chat.predict_and_parse(input=query)
print(res)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi how are you?
AI:  I am doing well, thank you for asking! How are you?
Human:
Human: can you write me a simple Python script that calculates the circumference
of a circle given a radius `r`
AI:  Sure, here's a Python script that calculates the circumference of a circle given a radius `r`:

```python
def circumference_of_circle(r):
    return 2*pi*r
```

You can use this function by calling it with the radius as an argument, like so:

```python
circumference_of_circle(3)  # returns 18.84955...
```

Hope that helps!
Human: Using this code I get the error:

`NameError: name 'pi' is not defined`

How can I fix this?
AI:  You need to import the math module at the beginning of your 

In [ ]:
# --- original function ---
def sum_numbers(n):
    total = 0
    for i in range(1, n+1):
        if i % 2 == 0:
            total += i
        else:
            total += i
    return total

# Test the function
result = sum_numbers(10)
print(result)


# --- refactored function ---
def sum_numbers(n):
    return sum([i for i in range(1, n+1)])

# Test the function
result = sum_numbers(10)
print(result)

With that we have our Falcon-40B powered chatbot running on a single GPU using ~27.3GB of GPU RAM.

---